In [1]:
# imports
import pandas as pd
from sqlalchemy import create_engine

In [5]:
# start Abbys section

In [9]:
target_df = pd.read_csv('target.csv', encoding='utf-8')
target_df.head()

,X.locale,Address.AddressLine1,Address.AddressLine2,Address.City,Address.CountryName,Address.County,Address.FormattedAddress,Address.IntersectionDescription,Address.Latitude,Address.Longitude,...,Is24Hours.Sat,IsOpen.Sat,Summary.Sat,ThruTime.Sat,BeginTime.Sun,Is24Hours.Sun,IsOpen.Sun,Summary.Sun,ThruTime.Sun,AllCapability
0,en-US,400 Oxford Exchange Blvd,NaN,Oxford,United States,Calhoun,"400 Oxford Exchange Blvd, Oxford, AL 36203-3459",SEC I-20 & Golden Creek Rd,33.608825,-85.783182,...,False,True,8:00 a.m.-11:00 p.m.,23:00:00,8:00:00,False,True,8:00 a.m.-10:00 p.m.,22:00:00,"['CVS pharmacy', 'CafÈ', 'Mobile Kiosk', 'Star..."
1,en-US,2640 Enterprise Dr,NaN,Opelika,United States,Lee,"2640 Enterprise Dr, Opelika, AL 36801-1511",NWC I-85 & Hwy 280,32.618108,-85.410636,...,False,True,8:00 a.m.-11:00 p.m.,23:00:00,8:00:00,False,True,8:00 a.m.-10:00 p.m.,22:00:00,"['CVS pharmacy', 'Cafe-Pizza', 'Fresh Grocery'..."
2,en-US,1129 S Park St,NaN,Carrollton,United States,Carroll,"1129 S Park St, Carrollton, GA 30117-4429",NWC US 27 & Hwy 166 Bypass,33.558920,-85.076473,...,False,True,8:00 a.m.-10:00 p.m.,22:00:00,8:00:00,False,True,8:00 a.m.-9:00 p.m.,21:00:00,NaN
3,en-US,555 Bullsboro Dr,NaN,Newnan,United States,Coweta,"555 Bullsboro Dr, Newnan, GA 30265-1045",SWC I-85 & Rte 34,33.392907,-84.758875,...,False,True,8:00 a.m.-11:00 p.m.,23:00:00,8:00:00,False,True,8:00 a.m.-10:00 p.m.,22:00:00,"['CVS pharmacy', 'CafÈ', 'Fresh Grocery', 'Mob..."
4,en-US,1591 Bradley Park Dr,NaN,Columbus,United States,Muscogee,"1591 Bradley Park Dr, Columbus, GA 31904-3071",NEC Bradley Pk & JR Allen Pkwy,32.534649,-84.971073,...,False,True,8:00 a.m.-11:00 p.m.,23:00:00,8:00:00,False,True,8:00 a.m.-10:00 p.m.,22:00:00,"['CVS pharmacy', 'CafÈ', 'Mobile Kiosk', 'Phot..."


In [17]:
removecolumns_target_df = target_df[['ID', 'Address.AddressLine1', 'Address.City', 'Address.PostalCode', 'Address.Subdivision']]
removecolumns_target_df.head()

,ID,Address.AddressLine1,Address.City,Address.PostalCode,Address.Subdivision
0,2153,400 Oxford Exchange Blvd,Oxford,36203-3459,AL
1,1499,2640 Enterprise Dr,Opelika,36801-1511,AL
2,1073,1129 S Park St,Carrollton,30117-4429,GA
3,1223,555 Bullsboro Dr,Newnan,30265-1045,GA
4,1179,1591 Bradley Park Dr,Columbus,31904-3071,GA


In [18]:
renamed_target_df = removecolumns_target_df.rename(columns={
    'ID': 'target_store_id',
    'Address.AddressLine1': 'street_address',
    'Address.City': 'city',
    'Address.PostalCode': 'zip_code',
    'Address.Subdivision': 'state_abr'
})
renamed_target_df.head()

,target_store_id,street_address,city,zip_code,state_abr
0,2153,400 Oxford Exchange Blvd,Oxford,36203-3459,AL
1,1499,2640 Enterprise Dr,Opelika,36801-1511,AL
2,1073,1129 S Park St,Carrollton,30117-4429,GA
3,1223,555 Bullsboro Dr,Newnan,30265-1045,GA
4,1179,1591 Bradley Park Dr,Columbus,31904-3071,GA


In [30]:
renamed_target_df['striped_zip'] = renamed_target_df['zip_code'].str.split('-', n = 1, expand = True)[0]

newzip_target_df = renamed_target_df.drop(columns=('zip_code'))
newzip_target_df

,target_store_id,street_address,city,state_abr,striped_zip
0,2153,400 Oxford Exchange Blvd,Oxford,AL,36203
1,1499,2640 Enterprise Dr,Opelika,AL,36801
2,1073,1129 S Park St,Carrollton,GA,30117
3,1223,555 Bullsboro Dr,Newnan,GA,30265
4,1179,1591 Bradley Park Dr,Columbus,GA,31904
...,...,...,...,...,...
1824,2239,22832 US Highway 281 N,San Antonio,TX,78258
1825,2429,135 Creekside Way,New Braunfels,TX,78130
1826,2438,700 Barnes Dr,San Marcos,TX,78666
1827,2725,5188 Kyle Center Dr,Kyle,TX,78640


In [3]:
# end Abbys section

In [4]:
# start Umar section

In [ ]:
# end Umar section